# Summarization of Research Papers
This document is meant to summarize the papers read during the Research Training, started on 30 July.
## 4CAC: 4-class classification of metagenome assemblies using machine learning and assembly graphs
Link: [https://www.biorxiv.org/content/10.1101/2023.01.20.524935v1.full.pdf](https://www.biorxiv.org/content/10.1101/2023.01.20.524935v1.full.pdf "‌")

### Evaluation criteria
* $ Precision = \frac{True Positive}{True Positive + False Positive} $ 

* $ Recall = \frac{True Positive}{True Positive + False Negative} $ 

* $ F1 Score = \frac{2 * Precision * Recall}{Precision + Recall} $ 

### Summary

The paper introduces a novel classifier called 4CAC that identifies 4 classes of microbial entities, namely, phages, plasmids, prokaryotes, and microeukaryotes, in metagenome assemblies simultaneously.

This system utilizes XGBoost algorithms and the assembly graph.

### Key Points

- While bacteria have been widely studied as the dominant species in the microbial communities, our understanding of phases, plasmids, and microeukaryotes lags behind _due to lower abundance._ The latter species are crucial in horizontal gene trasnfer and antibiotic resistance.
- The XGBoost classifier is trained and tested on all complete assemblies from the NCBI GenBank database. Several XGBoost models are trained for different sequence lengths.
- The classification is refined using the assembly graph, in which nodes represent contigs and edges represent subsequence overlaps between the corresponding contigs.
- 4CAC works well on _short-read_ assemblies.

### Potential Areas for Improvement

- Classification of underrepresented classes in real datasets lags behind other classifiers. Thus, more refined algorithms, e.g. Deep Learning models, can be explored to enhance the precision of 4CAC.
- 4CAC’s performance on long-reads can be improved.
- Graph-based approach can be refined.


## Terms and Definitions
**Genomics**: focuses on studying the entire genetic material () of a *single organism*

**Metagenomics**: focuses on studying the genetic material from *multiple organisms* found in a mixed sample

**Phage (Bacteriophage)**: specifically infects and replicates within bacteria. Phages play a key role in controlling bacterial populations in nature and can influence microbial ecosystems

**Horizontal Gene Transfer (HGT)**: the process by which an organism transfers genetics material to another organism that is not its offspring
* Why is HGT important?
⋅⋅* Spread of antibiotic resistance: bacteria can pass genes that confer resistance to antibiotics, even across species, making infections harder to treat
⋅⋅* Evolution and adaptation: HGT allows for rapid evolution, enabling organisms to quickly acquire traits that help them survive in new environments or under changing conditions 
* How is HGT done?
⋅⋅* Transduction: transfer of DNA between bacteria by viruses (phages)
⋅⋅* Conjugation: transfer of DNA through direct contact between 2 cells. **Plasmids** are small, circular DNA molecules that exist independently of a bacterial chromosome. A bacterium with a plasmid transfers it to another bacterium via a physical connection, called Pilus.
⋅⋅* Transformation: uptake of free DNA from the environment by a cell. **Microeukaryotes** may release or uptake free DNA in such a way 

**Vertical Gene Transfer (VGT)**: the process by which genes are transferred from parent to offspring through reproduction